# 小组成员
 1. 信息2301, 2310650118, 洪菁曼 （组长）
 2. 信息2302, 2310650219, 安雨琪

# 工作日志
## 问题一（安雨琪）
1. 2026-01-03 数据预处理 主要是将附件2各行重命名转换日期，附件一附件二通过单品编码的主键合并数据（数据库）
     * 预处理时发现反复运行合并数据会报错，删除已有重复列解决这个问题
2. 2026-01-03 基础分析 （1）将单品销售量前十列出 （2）6个品类年度分析、季度分析 （3）周末对比平常
     * 基础分析的一些方法不确定要不要做两个人讨论了一下决定从年、季度、周末入手
3. 2026-01-03 聚类分析 进行品类的聚类分组：分了三个类 、随机森林（回归分析）来说明各品类之间的相关关系；用spearman来进行销量和价格的相关性分析
     * 相关分析，一开始只采用了聚类分析和随机森林，发现描述的不够准确不太适合这类问题，进而选择用spearman和person。因为规律大概率不是线性的 所以用的spearman。spearman中一开始写的冗余代码太多一直在重复。后改进了写法，用封装的函数来优化代码

 智能商务数据分析实践  课程考核题目及要求

任务要求：
选题二选一完成，分组2~3人一组。
独立实践报告采用Jupyter写，最后在线提交notebook原文件以及其导出的PDF文件，无需包含原始数据，打包成ZIP包。
Jupyter笔记的的第一个Markdown单元格内必须写成员的班级、学号、姓名

报告的第二个Markdown单元格，写工作日志，至少4条以上，内容写标志性、自己做过努力的、跳过的坑。

1. 202x-xx-xx 对数据进行了...统计分析
1. 202x-xx-xx 补充了...分析
1. 202x-xx-xx 采用...交叉验证对选择模型..
1. ...
报告必须有相应的分析或说明文字，需要必要的完整中间过程（程序和输出结果）
报告最多分两级标题（章和节）
成员多于1人的小组，在报告每个第二级（节）的末尾以陈述方式说明成员的工作。

> 张三承担...完成...


选题2：蔬菜类商品的自动定价与补货决策问题

问题：
在生鲜商超中，一般蔬菜类商品的保鲜期都比较短，且品相随销售时间的增加而变差，大部分品种如当日未售出，隔日就无法再售。因此，商超通常会根据各商品的历史销售和需求情况每天进行补货。

由于商超销售的蔬菜品种众多、产地不尽相同，而蔬菜的进货交易时间通常在凌晨 3:00-4:00，为此商家须在不确切知道具体单品和进货价格的情况下，做出当日各蔬菜品类的补货决策。蔬菜的定价一般采用“成本加成定价”方法，商超对运损和品相变差的商品通常进行打折销售。可靠的市场需求分析，对补货决策和定价决策尤为重要。从需求侧来看，蔬菜类商品的销售量与时间往往存在一定的关联关系；从供给侧来看，蔬菜的供应品种在4月至10月较为丰富，商超销售空间的限制使得合理的销售组合变得极为重要。

附件1给出了某商超经销的6个蔬菜品类的商品信息；附件2和附件3分别给出了该商超2020年7月1日至2023年6月30日各商品的销售流水明细与批发价格的相关数据；附件4给出了各商品近期的损耗率数据。

请根据附件和实际情况建立数学模型解决以下问题：
问题1 蔬菜类商品不同品类或不同单品之间可能存在一定的关联关系，请分析蔬菜各品类及单品销售量的分布规律及相互关系。
问题2 考虑商超以品类为单位做补货计划，请分析各蔬菜品类的销售总量与成本加成定价的关系，并给出各蔬菜品类未来一周(2023 年 7 月 1-7 日)的日补货总量和定价策略，使得商超收益最大。
问题3 因蔬菜类商品的销售空间有限，商超希望进一步制定单品的补货计划，要求可售单品总数控制在 27-33 个，且各单品订购量满足最小陈列量 2.5 千克的要求。根据 2023 年 6 月 24-30 日的可售品种，给出 7 月 1 日的单品补货量和定价策略，在尽量满足市场对各品类蔬菜商品需求的前提助，请给出你们的意见和理由。

数据文件：
附件1 数据文件是6个蔬菜品类的商品信息
附件2 销售流水明细数据
附件3 蔬菜类商品的批发价格
附件4 蔬菜类商品的近期损耗率
附件说明：
附件1中部分单品名称包含的数字编号表示不同的供应来源。
附件4中的损耗率反映了近期商品的损耗情况，通过近期盘点周期的数据计算得到。


# 问题1
蔬菜类商品不同品类或不同单品之间可能存在一定的关联关系，请分析蔬菜各品类及单品销售量的分布规律及相互关系。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
import warnings
import os
warnings.filterwarnings('ignore')

# 设置中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
def smart_loader(file_path):
    """智能加载器：自动使用缓存版本"""
    # 构建缓存文件名（parquet格式）
    cache_file = os.path.splitext(file_path)[0] + '.parquet'
    
    # 如果缓存存在，直接读取
    if os.path.exists(cache_file):
        print("读取缓存文件...")
        return pd.read_parquet(cache_file)
    
    # 否则读取原始文件并创建缓存
    print("第一次读取，创建缓存...")
    # 根据文件扩展名选择读取方式
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        df = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    
    # 保存为parquet格式
    df.to_parquet(cache_file, engine='pyarrow')
    print(f"已创建缓存文件: {cache_file}")
    
    return df

# 使用方式
# df = smart_loader('your_file.xlsx')

In [3]:
try:
    df1 = smart_loader("./选题2-蔬菜自动定价与补货决策/附件1.xlsx")
    df2 = smart_loader("./选题2-蔬菜自动定价与补货决策/附件2.xlsx")
    df3 = smart_loader("./选题2-蔬菜自动定价与补货决策/附件3.xlsx")
    df4 = smart_loader("./选题2-蔬菜自动定价与补货决策/附件4.xlsx")
    print(f"附件1: {df1.shape}")
    print(f"附件2: {df2.shape}")
    print(f"附件3: {df3.shape}")
    print(f"附件4: {df4.shape}")
except Exception as e:
    print(f" 读取失败: {e}")
    exit()

读取缓存文件...
读取缓存文件...
读取缓存文件...
读取缓存文件...
附件1: (251, 4)
附件2: (878503, 7)
附件3: (55982, 3)
附件4: (6, 3)


In [4]:
#数据预处理
data = df2.copy()
data.columns = ['销售日期', '扫码时间', '单品编码', '销量', '单价', '销售类型', '是否打折']

# 转换日期
data['日期'] = pd.to_datetime(data['销售日期'])
data['年份'] = data['日期'].dt.year
data['季度'] = data['日期'].dt.quarter
data['月份'] = data['日期'].dt.month
data['星期'] = data['日期'].dt.weekday
data['是否周末'] = data['星期'] >= 5

In [6]:
columns_to_drop = ['分类名称_x', '分类名称_y', '分类名称_data', '分类名称_df1', '单品名称_x', '单品名称_y', '单品名称_data', '单品名称_df1']
for col in columns_to_drop:
    if col in data.columns:
        data = data.drop(columns=[col])
if '单品编码' in df1.columns and '分类名称' in df1.columns and '单品名称' in df1.columns:
# 合并数据，将单品名称也纳入合并范围
    data = pd.merge(data, df1[['单品编码', '分类名称', '单品名称']], on='单品编码', how='left', suffixes=('_data', '_df1'))
    print(data.columns)
    data['品类'] = data['分类名称_data'].fillna(data['分类名称_df1'])
    data['单品名称'] = data['单品名称_data'].fillna(data['单品名称_df1'])

    # 选择需要的列，同时包含单品名称
    data = data[['单品编码', '单品名称', '品类', '销量', '单价','是否打折', '日期', '年份', '季度','月份', '星期', '是否周末']]
else:
    print("附件1没有品类信息或单品名称信息，使用单品编码前2位作为品类")
    data['品类'] = data['单品编码'].astype(str).str[:2]
    # 如果没有单品名称信息，单品名称列将填充缺失值
    data['单品名称'] = None
    data = data[['单品编码', '单品名称', '品类', '销量', '单价', '是否打折', '日期', '年份', '季度','月份', '星期', '是否周末']]

print(f"处理完成: {len(data)} 行数据")
data.head()

Index(['销售日期', '扫码时间', '单品编码', '销量', '单价', '销售类型', '是否打折', '日期', '年份', '季度',
       '月份', '星期', '是否周末', '分类名称_data', '单品名称_data', '分类名称_df1', '单品名称_df1'],
      dtype='object')
处理完成: 878503 行数据


,单品编码,单品名称,品类,销量,单价,是否打折,日期,年份,季度,月份,星期,是否周末
0,102900005117056,泡泡椒(精品),辣椒类,0.396,7.6,否,2020-07-01,2020,3,7,2,False
1,102900005115960,大白菜,花叶类,0.849,3.2,否,2020-07-01,2020,3,7,2,False
2,102900005117056,泡泡椒(精品),辣椒类,0.409,7.6,否,2020-07-01,2020,3,7,2,False
3,102900005115823,上海青,花叶类,0.421,10.0,否,2020-07-01,2020,3,7,2,False
4,102900005115908,菜心,花叶类,0.539,8.0,否,2020-07-01,2020,3,7,2,False


## 基础分析
单品销量前十

In [7]:
sales_by_cat = data.groupby('单品名称')['销量'].sum().sort_values(ascending=False)
for cat, sales in sales_by_cat.head(10).items():  # 只显示前10
    print(f"  {cat}: {sales:,.1f}千克")

  芜湖青椒(1): 28,164.3千克
  西兰花: 27,537.2千克
  净藕(1): 27,149.4千克
  大白菜: 19,187.2千克
  云南生菜: 15,910.5千克
  金针菇(盒): 15,596.0千克
  云南生菜(份): 14,325.0千克
  紫茄子(2): 13,602.0千克
  西峡香菇(1): 11,920.2千克
  小米椒(份): 10,833.0千克


In [8]:
#各品类销售额
data['销售额'] = data['销量'] * data['单价']

# 按年份和分类汇总销量和销售额
yearly = data.groupby(['年份', '品类']).agg({
    '销量': 'sum',
    '销售额': 'sum',
}).reset_index()

print("\n各品类年度销量和销售额:")
years = sorted(yearly['年份'].unique())
for year in years:
    year_data = yearly[yearly['年份'] == year]
    print(f"\n{year}年:")
    print(f"{'分类':<8} {'销量(kg)':>12} {'销售额(元)':>15}")
    for _, row in year_data.sort_values('销售额', ascending=False).head(6).iterrows():
        print(f"{row['品类']:<8} {row['销量']:12,.0f} {row['销售额']:15,.0f}")

# 年度增长率（销量和销售额）
print("\n年度增长率（销量和销售额）:")
top_cats = yearly.groupby('品类')['销售额'].sum().nlargest(6).index

for cat in top_cats:
    cat_data = yearly[yearly['品类'] == cat].sort_values('年份')
    if len(cat_data) >= 2:
        first_year = cat_data.iloc[0]
        last_year = cat_data.iloc[-1]
        
        # 销量增长率
        sales_growth = (last_year['销量'] - first_year['销量']) / first_year['销量'] * 100 if first_year['销量'] > 0 else 0
        
        # 销售额增长率
        revenue_growth = (last_year['销售额'] - first_year['销售额']) / first_year['销售额'] * 100 if first_year['销售额'] > 0 else 0
        
    
        
        print(f"\n{cat}:")
        print(f"  销量增长率（2020-2023）: {sales_growth:+.1f}% ({first_year['销量']:,.0f}kg → {last_year['销量']:,.0f}kg)")
        print(f"  销售额增长率（2020-2023）: {revenue_growth:+.1f}% (¥{first_year['销售额']:,.0f} → ¥{last_year['销售额']:,.0f})")



各品类年度销量和销售额:

2020年:
分类             销量(kg)          销售额(元)
花叶类            39,322         234,439
食用菌            14,654         138,142
辣椒类            12,187         125,532
花菜类             9,431          84,183
水生根茎类           6,199          52,632
茄类              4,790          34,601

2021年:
分类             销量(kg)          销售额(元)
花叶类            58,682         364,897
辣椒类            23,238         236,387
食用菌            21,921         206,950
水生根茎类          13,103         116,849
花菜类            12,688         112,079
茄类              7,796          63,201

2022年:
分类             销量(kg)          销售额(元)
花叶类            67,105         308,505
辣椒类            34,680         247,009
食用菌            23,346         168,546
水生根茎类          15,754         131,482
花菜类            14,158         122,287
茄类              6,259          58,943

2023年:
分类             销量(kg)          销售额(元)
花叶类            33,412         171,229
辣椒类            21,484         145,205
食用菌            16,165         105,960
花菜类 

In [9]:
quarterly = data.groupby(['季度', '品类']).agg({
    '销量': 'mean'  # 季度日均销量
}).reset_index()

quarter_names = {1: 'Q1(1-3月)', 2: 'Q2(4-6月)', 3: 'Q3(7-9月)', 4: 'Q4(10-12月)'}

print("\n各季度平均日销量 (kg):")
for quarter in [1, 2, 3, 4]:
    quarter_data = quarterly[quarterly['季度'] == quarter]
    print(f"\n{quarter_names[quarter]}:")
    for _, row in quarter_data.sort_values('销量', ascending=False).head(3).iterrows():
        print(f"  {row['品类']}: {row['销量']:.1f}kg/天")

# 季节性分析
print("\n季节性分析（旺季/淡季）:")
for cat in quarterly['品类'].unique()[:6]:
    cat_data = quarterly[quarterly['品类'] == cat]
    peak = cat_data.loc[cat_data['销量'].idxmax()]
    low = cat_data.loc[cat_data['销量'].idxmin()]
    diff = (peak['销量'] - low['销量']) / low['销量'] * 100
    print(f"  {cat}: 旺季{quarter_names[peak['季度']]}, 淡季{quarter_names[low['季度']]}, 差异{diff:.0f}%")


各季度平均日销量 (kg):

Q1(1-3月):
  水生根茎类: 0.8kg/天
  花叶类: 0.6kg/天
  食用菌: 0.6kg/天

Q2(4-6月):
  花叶类: 0.5kg/天
  食用菌: 0.5kg/天
  水生根茎类: 0.5kg/天

Q3(7-9月):
  水生根茎类: 0.7kg/天
  花叶类: 0.6kg/天
  茄类: 0.5kg/天

Q4(10-12月):
  水生根茎类: 0.7kg/天
  花叶类: 0.6kg/天
  食用菌: 0.5kg/天

季节性分析（旺季/淡季）:
  水生根茎类: 旺季Q1(1-3月), 淡季Q2(4-6月), 差异58%
  花叶类: 旺季Q1(1-3月), 淡季Q2(4-6月), 差异18%
  花菜类: 旺季Q1(1-3月), 淡季Q3(7-9月), 差异4%
  茄类: 旺季Q3(7-9月), 淡季Q1(1-3月), 差异8%
  辣椒类: 旺季Q1(1-3月), 淡季Q3(7-9月), 差异24%
  食用菌: 旺季Q1(1-3月), 淡季Q3(7-9月), 差异39%


In [10]:
#周末对比平常

data['是否周末'] = data['是否周末'].astype(bool)

# 按品类和是否周末分组，计算每组销量的均值
grouped = data.groupby(['品类', '是否周末'])['销量'].mean().reset_index()

# 重塑数据，将是否周末作为列名
pivot_table = grouped.pivot(index='品类', columns='是否周末', values='销量')
# 重命名列名，便于识别
pivot_table.columns = ['平时销量均值', '周末销量均值']

pivot_table['周末/平时销量均值比值'] = pivot_table['周末销量均值'] / pivot_table['平时销量均值']
pivot_table['销量均值差值'] = pivot_table['周末销量均值'] - pivot_table['平时销量均值']
pivot_table['周末还是平时卖得好'] = pivot_table['周末/平时销量均值比值'].apply(
    lambda x: '周末' if x > 1 else '平时'
)
# 打印结果
print(pivot_table)

         平时销量均值    周末销量均值  周末/平时销量均值比值    销量均值差值 周末还是平时卖得好
品类                                                        
水生根茎类  0.688026  0.698699     1.015513  0.010673        周末
花叶类    0.600892  0.592800     0.986532 -0.008093        平时
花菜类    0.479111  0.488519     1.019636  0.009408        周末
茄类     0.498880  0.500941     1.004131  0.002061        周末
辣椒类    0.440589  0.439886     0.998405 -0.000703        平时
食用菌    0.512579  0.512721     1.000277  0.000142        周末


In [11]:
from scipy import stats

# 对原有的数据分组聚合逻辑添加日志输出，方便用户了解函数功能和数据处理进展
def summarize_data(data):
    #对数据按日期和品类进行汇总，计算每个品类每天的总销量、平均单价和总销售额。
    daily_summary = data.groupby(['日期', '品类']).agg({
        '销量': 'sum',
        '单价': 'mean',
        '销售额': 'sum'
    }).reset_index()
    return daily_summary

# 封装相关性计算逻辑，让代码结构更清晰，便于复用和测试
def calculate_spearman_correlation(daily_summary):
    #计算每个品类价格与销量的Spearman相关系数，并分析相关性强度、显著性和方向。
    spearman_results = []
    for cat in daily_summary['品类'].unique():
        cat_data = daily_summary[daily_summary['品类'] == cat]
        if len(cat_data) >= 10:  # 至少需要10天数据
            # 计算Spearman相关系数（更稳健，适用于非线性关系）
            price_sales_corr, price_sales_p = stats.spearmanr(cat_data['单价'], cat_data['销量'])
            # 判断相关性强度
            strength = get_correlation_strength(price_sales_corr)
            # 判断显著性 (p < 0.05 为显著相关)
            significant = "显著" if price_sales_p < 0.05 else "不显著"
            direction = "负相关" if price_sales_corr < 0 else "正相关"
            spearman_results.append({
                '品类': cat,
                'Spearman相关系数': round(price_sales_corr, 3),
                'p值': round(price_sales_p, 3),
                '显著性': significant,
                '相关性强度': strength,
                '方向': direction,
                '数据天数': len(cat_data)
            })
    return pd.DataFrame(spearman_results).sort_values('Spearman相关系数')

# 将相关性强度判断逻辑封装成函数，提高代码的复用性和可维护性
def get_correlation_strength(corr):
    #根据相关系数判断相关性强度。
    #param corr: 相关系数
    
    if abs(corr) > 0.5:
        return "强"
    elif abs(corr) > 0.3:
        return "中等"
    elif abs(corr) > 0.1:
        return "弱"
    else:
        return "无"

# 封装结果打印逻辑，使代码更模块化，便于代码的修改和扩展
def print_correlation_results(spearman_df):
    #打印各品类价格与销量的Spearman相关性分析结果。
    print("\n各品类价格与销量Spearman相关性分析:")
    print(f"{'品类':<10} {'Spearman相关系数':>15} {'p值':>10} {'显著性':>8} {'强度':>8} {'方向':>8}")
    for _, row in spearman_df.iterrows():
        print(f"{row['品类']:<10} {row['Spearman相关系数']:15.3f} {row['p值']:10.3f} "
              f"{row['显著性']:>8} {row['相关性强度']:>8} {row['方向']:>8}")

# 分析显著相关的品类，并打印结果，使代码结构更清晰，功能模块化
def analyze_significant_correlations(spearman_df):
    print("\n显著相关的品类 (p < 0.05):")
    significant_corr = spearman_df[spearman_df['显著性'] == '显著']
    if len(significant_corr) > 0:
        # 强负相关
        strong_neg = significant_corr[(significant_corr['Spearman相关系数'] < -0.5)]
        # 中等负相关
        medium_neg = significant_corr[(significant_corr['Spearman相关系数'] >= -0.5) &
                                      (significant_corr['Spearman相关系数'] < -0.3)]
        # 弱负相关
        weak_neg = significant_corr[(significant_corr['Spearman相关系数'] >= -0.3) &
                                    (significant_corr['Spearman相关系数'] < -0.1)]
        # 弱正相关
        weak_pos = significant_corr[(significant_corr['Spearman相关系数'] >= 0.1) &
                                    (significant_corr['Spearman相关系数'] < 0.3)]
        # 中等正相关
        medium_pos = significant_corr[(significant_corr['Spearman相关系数'] >= 0.3) &
                                      (significant_corr['Spearman相关系数'] < 0.5)]
        # 强正相关
        strong_pos = significant_corr[(significant_corr['Spearman相关系数'] >= 0.5)]

        print_correlation_category(strong_neg, "强负相关")
        print_correlation_category(medium_neg, "中等负相关")
        print_correlation_category(weak_neg, "弱负相关")
        print_correlation_category(weak_pos, "弱正相关")
        print_correlation_category(medium_pos, "中等正相关")
        print_correlation_category(strong_pos, "强正相关")
    else:
        print("无显著相关的品类。")

# 封装打印特定相关性类别品类的逻辑，减少代码重复，提高代码的可维护性
def print_correlation_category(category_df, category_name):
    if not category_df.empty:
        print(f"\n{category_name}:")
        for _, row in category_df.iterrows():
            print(f"  {row['品类']}: r={row['Spearman相关系数']:.3f}, p={row['p值']:.3f}")

# 主函数
def main(data):
    daily_summary = summarize_data(data)
    spearman_df = calculate_spearman_correlation(daily_summary)
    print_correlation_results(spearman_df)
    analyze_significant_correlations(spearman_df)

main(data)


各品类价格与销量Spearman相关性分析:
品类            Spearman相关系数         p值      显著性       强度       方向
食用菌                 -0.461      0.000       显著       中等      负相关
水生根茎类               -0.442      0.000       显著       中等      负相关
辣椒类                 -0.312      0.000       显著       中等      负相关
茄类                  -0.299      0.000       显著        弱      负相关
花菜类                 -0.272      0.000       显著        弱      负相关
花叶类                 -0.160      0.000       显著        弱      负相关

显著相关的品类 (p < 0.05):

中等负相关:
  食用菌: r=-0.461, p=0.000
  水生根茎类: r=-0.442, p=0.000
  辣椒类: r=-0.312, p=0.000

弱负相关:
  茄类: r=-0.299, p=0.000
  花菜类: r=-0.272, p=0.000
  花叶类: r=-0.160, p=0.000
